# COMP41680 API Assignment 1
## Robert Keenan 15333066
### Using the Weather and Airport flight statuses to determine a relationship between them

## Step 1: APIs
For the first API, I have chosen to use the Aviationstack API which is one of the largest and most utilised APIs for real time and historical flight data. I will use this in unison with a weather API which will be used to determine the correlation/relationship between the weather and flight's statuses at different airports. I will then further investigate whether the weather affects different airlines, routes or airplane models differently such as between a small regional jet and a large, long range jet such as the Boeing 777.
- The Aviationstack API can be found here: https://aviationstack.com
- The weather API can be found here at: TODO/FIXME

I received an API key for my Aviationstack account which is given as b4a871dbf85b4f670dafbdc9615a68e9.

I will firstly import any libraries or packages needed and then make a start on the data collection stage of the assignment. 



In [ ]:
import requests

params = {
  'access_key': 'b4a871dbf85b4f670dafbdc9615a68e9'
}
